In [ ]:
import time
from glob import glob

import numpy as np
from PIL import Image

import model

In [ ]:
# 运行前需要的配置
# 1. 修改 angle/predict.py 32行 模型地址
# TODO 2. 修改 /ctpn/ctpn/model.py 36 行 checkpoint 地址
# TODO 3. 覆盖/ctpn_detect_v1/ctpn/lib/utils

# https://tesseract-ocr.github.io/tessdoc/ImproveQuality

im = Image.open("/Users/liuliangjun/Downloads/test18.png")
img = np.array(im.convert('RGB'))

t = time.time()

result, img, text_recs = model.model(
    img, model='crnn', adjust=False, detectAngle=True)

print("It takes time:{}s".format(time.time() - t))
print("---------------------------------------")

# 对版面进行排序（bubbleSort）
# [x1,y1] [x2,y2]
benchmark = 8  # 基准像素点，相差在这个范围内的文本域都认为是在同一水平线上

arr = sorted(result.values(), key=lambda x:x[0][1])  # 按左上角点y排序
arr = [[*row, idx] for idx, row in enumerate(result.values())] # 每个文本域预先设定一个idx(idx理解为所在的行号)

_len = len(arr)
for i in range(_len):
    for j in range(0, _len-i-1):
        if (arr[j+1][0][1] - arr[j][0][1]) <= benchmark and arr[j][0][0] > arr[j+1][0][0]:
            # 如果 (y2-y1)<= benchmark and x1 > x2 则交换它们的位置
            arr[j], arr[j+1] = arr[j+1], arr[j]
        if (arr[j+1][0][1] - arr[j][0][1]) <= benchmark:
            # 如果 (y2-y1)<= benchmark 则后者继承前者的idx（也即认为他们在同一水平线上）
            arr[j+1][-1] =  arr[j][-1]
            
            
# 将同一水平线的文本域合并
res = {}
for v in arr:
    res.setdefault(v[-1], []).append(v[1])

# 按行打印
for k, v in res.items():
    print(k, v)
            
Image.fromarray(img)

In [ ]:
# confidence低于80 则需要匹配(只匹配单词，数字和符号不考虑)
import Levenshtein

word_list = ['Hong','Kong', 'Items']

def correct(word):
    """
    与word_list中的单词比较编辑距离，小于等于1的就返回，否则返回原词
    """
    print(word)
    data = [(w, Levenshtein.distance(word, w)) for w in word_list]
    for w, d in sorted(data, key=lambda x: x[1]):
        if d <= 1:
            return w
        break
    return word

correct('Hongs')
    

In [ ]:
# 包含 MEDICAL / CENTRE  / hospital 等则认为这是医院名 一般在开始几行
# Building street road Room  Hong Kong  st

# 按行打印
data = []
for idx, line in res.items():
    line_data = []
    for box in line:
        val = [correct(str(word)) if (str(word).isalpha() and confidence < 80) else str(word) for confidence, word in box]
        line_data.append(' '.join(val))
    data.append(line_data)
data

In [ ]:
# 匹配 hospital name, address
text1 = '\n'.join(sum(res.values(), []))
hospital_name = regex.findall('(.*(MEDICAL|CENTRE|HOSPITAL).*)', text1, flags=regex.IGNORECASE)
address = regex.findall('\n([\w,&]*(Room|Office|Building|Street|Road).*)', text1, flags=regex.IGNORECASE)


# 匹配tel,fax,transaction code, date, Doctor, Patient Code, Patient Name, total, Paid, Diagnosis
import regex 
text2 = '\n'.join([' '.join(v) for k, v in res.items()])  
# print(text2)
print('Result'.center(50, '-'))
regex_map = {
    'Tel': '(Tel : ([+\d ]+)){e<=1}',
    'Fax': '(Fax : ([+\d ]+)){e<=1}',
    'Transaction Code': '(Transaction Code : ([\d]+)){e<=2}',
    'Date': '(Date : (\d{2} [a-zA-Z,]+ \d{4})){e<=2}',
    'Doctor': '(Doctor : (\w+)\n){e<=2}',
    'Patient Code': '(Patient Code : (\d+)\n){e<=2}',
    'Patient Name': '(Patient Code : \d+\n(.*)\nItems){e<=3}',
    'Total': '(Total : ([\d\.\,]+)){e<=2}',
    'Paid': '(Paid : ([\d\.\,]+)){e<=2}',
    'Diagnosis': '(Diagnosis : (.*)\n){e<=3}'
}
# print(hospital_name)
print(f"Hospital Name: {hospital_name[0][0] if hospital_name else ''}")
# print(address)
print(f"Address: {address[0][0] if address else ''}")
for k, pat in regex_map.items():
    ret = regex.findall(pat, text2)
    print(f"{k}: {ret[0][-1] if ret else ''}")
    

from pandas import DataFrame
start_idx, end_idx = 0, 0
values = list(res.values())
for k, v in enumerate(values):
    to_str = ' '.join(v)
    if regex.match('(Items.*Amount){e<=2}', to_str):
        start_idx = k
    elif regex.match('(Total.*Paid){e<=2}', to_str):
        end_idx = k
        
data = values[start_idx:end_idx]
DataFrame(data[1:], columns=data[0])

In [ ]:
# result, img, text_recs

In [ ]:
# result
img.shape

xDim, yDim = img.shape[1], img.shape[0]
print(xDim, yDim)